# feature_selection

In [1]:
import pandas as pd
import numpy as np

In [2]:
import nbimporter

from pre_processing import load_appended_dataset, separate_train_test, load_datasets, load_featured_datasets, load_featured_appended_dataset 

Importing Jupyter notebook from pre_processing.ipynb
Importing Jupyter notebook from feature_generation.ipynb


# Seleccion de K Mejores Features

In [5]:
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectKBest

In [6]:
train,test = load_featured_datasets()

In [7]:
train.fillna(train.mean(),inplace=True)

In [8]:
X = train.drop('precio', axis=1).values
Y = train['precio'].values

In [9]:
X.shape

(240000, 136)

In [10]:
Selector = SelectKBest(f_regression, k=50)

In [11]:
Selector.fit(X,Y)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equa

SelectKBest(k=50, score_func=<function f_regression at 0x7f29add1a7b8>)

In [12]:
def k_features_selector(k_features,train_df):
    train_na_filled_df = train_df.fillna(train_df.mean())
    X = train_na_filled_df.drop('precio', axis=1).values
    Y = train_na_filled_df['precio'].values
    Selector = SelectKBest(f_regression, k=k_features)
    Selector.fit(X,Y)
    return Selector